# Multiple Knapsack Problem

Lösungen des MKP werden in einer Liste dargestellt. Jedes Item hat eine ID, die den Index in dieser Liste darstellt. Den Wert an dieser Stelle entspricht der Knapsack-ID, sodass z.B. folgende Lösung [1, 1, -1, 2, 0] bedeutet, dass folgenden Zuordnung getroffen wurde:
- Knapsack 0: Item mit ID 4
- Knapsack 1: Items mit ID 0 und 1
- Knapsack 2: Item mit ID 3
Das Item mit ID 2 wurde nicht zugeordnet und damit auch in keinen Knapsack gepackt.

Diese Repräsentation hat den Vorteil, dass man möglichst viel von der Codebasis aus dem Seminar benutzen kann, man muss nur kleinere Änderungen machen. Insbesondere habe ich überall Type Hints hinzugefügt und eine Klassen zu Dataclasses gemacht, was das Weiterbenutzen des Codes viel leichter macht und den Code auch verständlicher macht.

Ein Lösungsobjekt kann einfach aus der Listendarstellung erzeugt werden und besitzt folgende Methoden und Attribute:
- `allocation`: Die Liste, die die Zuordnungen zwischen Item und Knapsack enthält
- `penalty`: Jedes restliche Volumen im Knapsack wird bestraft. Die Summe der Strafen ist in `penalty` gespeichert.
- `earnings`: Jedes Item ist unterschiedlich viel Wert. Sie Summe der Werte der einzelnen mitgenommenen Items wird in `earnings` gespeichert.
- `profit`: `earnings - penalty`
- `Solution.printSolution()` gibt eine übersichtlichere Darstellung der Lösung aus
- `Solution.to_json(filename: str, includeEarnings = False, includePenalty = False, force = False)` schreibt die Lösung im JSON-Format in die angegebene Datei. Es wird überprüft ob die Datei schon exisiert, wenn ja, wird diese nicht überschrieben, es sei denn `force = True`. Die Earnings und Penalties werden normalerweise nicht mit abgespeichert, dies lässt sich mittels `includeEarnings` und `includePenalty` ändern.

In [5]:
from cls.Solution import Solution
from cls.InputData import InputData
from cls.EvaluationLogic import EvaluationLogic

import os

data = InputData(os.path.join(os.getcwd(), "Testinstanzen", "Instance01_m3_n20.json"))
evalutation = EvaluationLogic(data)

sol = Solution([2, 2, -1, 1, 1, 1, 2, 0, -1, 2, 2, -1, 0, 1, 1, -1, 2, 2, -1, 1])
evalutation.calcProfit(sol)
sol.printSolution()
print(sol)

Knapsack 0: [7, 12]
Knapsack 1: [3, 4, 5, 13, 14, 19]
Knapsack 2: [0, 1, 6, 9, 10, 16, 17]
Solution(allocation=[2, 2, -1, 1, 1, 1, 2, 0, -1, 2, 2, -1, 0, 1, 1, -1, 2, 2, -1, 1], penalty=136, earnings=225, profit=89)


Um eine erste konstruktive Lösung zu ermitteln, wird versucht die Knapsacks nacheinander mit Items zu füllen, bis ein Knapsack voll ist. Dann wird der nächste Knapsack gefüllt, usw. Um die Lösung gleich ein bisschen zu verbessern, werden die Items nach Profit/Weight absteigend sortiert, es werden also bevorzugt kleine und wertvolle Items mitgenommen. Weiterhin werden die Knapsacks nach Penalty absteigend sortiert, damit der erste Knapsack möglichst voll wird und wenig Strafkosten generiert (am Anfang werden ja insbesondere kleine Items mitgenommen, damit lässt sich der Platz gut füllen).

Die so gefundene Lösung wird evaluiert und in den SolutionPool hinzugefügt.

In [7]:
from cls.ConstructiveHeuristic import ConstructiveHeuristics
from cls.SolutionPool import SolutionPool

pool = SolutionPool()
heuristik = ConstructiveHeuristics(EvaluationLogic(data), pool)
heuristik.Run(data, "greedy")
print(pool)

Generating an initial solution according to greedy.
SolutionPool(Solutions=[Solution(allocation=[2, 2, -1, 1, -1, 1, 2, 0, 1, 2, 2, -1, 0, 1, 1, -1, 2, 2, -1, 1], penalty=159, earnings=241, profit=82)])


Für die Nachbarschaften musste ich dann immer nur Überprüfen, ob eine Lösung gültig ist oder nicht, deswegen gibt `EvaluationLogic.calcProfit()` auch immer einen Boolean zurück, ob die Lösung gültig ist. Ist eine Lösung nicht gültig, so wird diese nicht den MoveSolutions hinzugefügt. Es kann natürlich sein, dass eine Nachbarschaft mal gar keine gültige Lösung erzeugen kann, aber die nächste Nachbarschaft aus `BaseNeighborhood.MoveSolutions` die beste Lösung haben will. Im Fall, dass dieser leer ist, wird die beste Lösung aus dem SolutionPool zurückgegeben, indem ja mindestens immer die gültige konstruktive Lösung liegt.

Es sind folgende Nachbarschaften implementiert:
- Swap: Tauscht zwei Items
- Insertion: Fügt ein Item mit einer bestimmten ID in einen Knapsack mit einer bestimmten ID ein
- BlockMoveK3: (kommt aus dem letzten Semester)
- TwoEdgeExchange: tauscht einige Items gleichzeitig aus

Nach meinen bisherigen Erfahrungen erzeugen BlockMoveK3 und TwoEdgeExchange nur selten gültige Lösungen.

In [8]:
from cls.Neighborhood import SwapNeighborhood

swapN = SwapNeighborhood(data, pool.GetLowestProfitSolution().allocation, EvaluationLogic(data), pool)
swapN.DiscoverMoves()
swapN.EvaluateMoves("BestImprovement")
print(swapN.MakeBestMove())


Solution(allocation=[2, 2, -1, 1, 1, 1, 2, 0, -1, 2, 2, -1, 0, 1, 1, -1, 2, 2, -1, 1], penalty=136, earnings=225, profit=89)


Die Tabu Search ist ein ImprovementAlgorithm, der eine gewissen Anzahl an Iterationen durchlaufen kann. Es wird zu einer Startlösung mittels IterativeImprovement eine Nachbarschaft aufgebaut und die beste Lösung in dieser Nachbarschaft rausgesucht. Ist diese Nachbarschaft allerdings in der Tabu Liste, so wird diese nicht angenommen, es sei denn, sie erfüllt ein Aspirationskriterium (besser als aktuell beste Lösung). Ich habe mich für ein langfristiges Gedächtnis entschieden, das war am leichtesten zu implmentieren, da nie Elemente aus der Tabu Liste entfernt werden müssen.

In [9]:
from cls.Solver import Solver
from cls.ImprovementAlgorithm import TabuSearch

solver = Solver(data, 42)
tabu = TabuSearch(data, 10, neighborhoodEvaluationStrategy = "BestImprovement", neighborhoodTypes = ['Swap', 'Insertion', 'BlockK3', 'TwoEdgeExchange'])
bestSol = solver.RunLocalSearch("greedy", tabu)

Generating an initial solution according to greedy.
Constructive solution found.
Solution(allocation=[2, 2, -1, 1, -1, 1, 2, 0, 1, 2, 2, -1, 0, 1, 1, -1, 2, 2, -1, 1], penalty=159, earnings=241, profit=82)
Best found Solution.
Solution(allocation=[2, 2, -1, 1, 1, 1, 2, 0, -1, 2, 2, -1, 0, 1, 1, -1, 2, 2, -1, 1], penalty=136, earnings=225, profit=89)
